We have 8 csv files:

- Bullying_1: aggression_parsed_dataset.csv       (115864 rows × 5 columns)
- Bullying_2: attack_parsed_dataset.csv           (115864 rows × 5 columns) 
- Bullying_3: twitter_parsed_dataset.csv          (16851 rows × 5 columns) 
- Bullying_4: twitter_racism_parsed_dataset.csv   (13471 rows × 5 columns) 
- Bullying_5: twitter_sexism_parsed_dataset.csv   (14881 rows × 5 columns) 
- Bullying_6: youtube_parsed_dataset.csv          (3464 rows × 10 columns) 
- Bullying_7: toxicity_parsed_dataset.csv         (159686 rows × 5 columns) 
- Bullying_8: kaggle_parsed_dataset.csv           (8799 rows × 4 columns) 


https://www.kaggle.com/datasets/saurabhshahane/cyberbullying-dataset?select=aggression_parsed_dataset.csv

In [1]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import pickle
import joblib

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from imblearn.over_sampling import SMOTE

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix, hamming_loss, classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder


import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)



In [2]:
# Loading the data

B_types = pd.read_csv("B_types.csv", index_col=0)

In [3]:
# Original dataframe
B_types

,Text,oh_label,type_label
0,fuck you bitch. and fuck you lousy and misle...,1.0,aggresion
1,RT @ryanklang: @Norse_Gamer @MT8_9 @ActionFlic...,1.0,sexism
2,::Hopefully one day the editors of Wikipedia ...,1.0,toxicity
3,::::What a deceptive little creature you are...,1.0,aggresion
4,lol smalljim. u have small penis. jk lol,1.0,attack
...,...,...,...
35195,I FUCKING HATE NIGGERS AND SPICS AND JEWS AND...,1.0,toxicity
35196,` == Congratulations == Well I suppose I hav...,1.0,aggresion
35197,"== You too! == You're also gonna get it, di...",1.0,attack
35198,Stop fucking harrassing me!,1.0,attack


In [4]:
B_types['type_label'] = B_types['type_label'].replace('aggresion', 'aggression')

In [5]:
B_types['type_label'].value_counts()

type_label
aggression    10000
toxicity      10000
attack        10000
sexism         3300
racism         1900
Name: count, dtype: int64

In [6]:
# sample of aggression
B_types.loc[B_types["type_label"]=='aggression'].sample()

,Text,oh_label,type_label
12828,"I swear, you're extremely stupid and oblivious...",1.0,aggression


In [7]:
# sample of attack
B_types.loc[B_types["type_label"]=='attack'].sample()

,Text,oh_label,type_label
21421,I have a Ph.D so I think I am better than ever...,1.0,attack


In [8]:
# sample of racism
B_types.loc[B_types["type_label"]=='racism'].sample()

,Text,oh_label,type_label
23013,@Lithobolos @ZaibatsuNews I don't care what th...,1.0,racism


In [9]:
# sample of sexism
B_types.loc[B_types["type_label"]=='sexism'].sample()

,Text,oh_label,type_label
16662,RT @AlexNew93 I'm not sexist but I just cannot...,1.0,sexism


In [10]:
# sample of toxicity
B_types.loc[B_types["type_label"]=='toxicity'].sample()

,Text,oh_label,type_label
27122,` == Stop your fucking spamming == I have to...,1.0,toxicity


### Text Preprocessing

In [11]:
### Text Preprocessing:
# Cleaning and preprocessing the text data. This involves tasks like removing stop words, handling emojis, 
# and converting text to lowercase.
# Tokenizing the text to convert it into a format suitable for machine learning algorithms.

In [12]:
# checking null values
B_types.isna().sum()

Text          0
oh_label      0
type_label    0
dtype: int64

In [13]:
# Preprocessing the text. Cenverting to lower_case, removing any special characters/symbols and removing extra spaces.

def clean_text(text):
    text = text.lower() # Convert to lowercase
    text = re.sub(r'\W', ' ', text) # Remove special characters and symbols
    text = re.sub(r'\s+', ' ', text, flags=re.MULTILINE) # Remove extra spaces
    words = [word for word in text.split() if word.isalnum() and word.isalpha()] # Split the text into words and keep only alphanumeric words
    cleaned_text = ' '.join(words) # Join the words with a space
    return cleaned_text

In [14]:
# Apply the clean_text function to the "Text" column
B_types['Cleaned_Text'] = B_types['Text'].apply(clean_text)
B_types[['Cleaned_Text']]

,Cleaned_Text
0,fuck you bitch and fuck you lousy and misleadi...
1,rt ryanklang actionflickdoc hey look me too ht...
2,hopefully one day the editors of wikipedia wil...
3,what a deceptive little creature you are indee...
4,lol smalljim u have small penis jk lol
...,...
35195,i fucking hate niggers and spics and jews and ...
35196,congratulations well i suppose i have to confe...
35197,you too you re also gonna get it dickface
35198,stop fucking harrassing me


In [15]:
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.data.path.append('/root/nltk_data/corpora/')

In [16]:
# Tokenizing and removing stopwords

def tokenizing_and_stopwords(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    text = " ".join(filtered_tokens)
    return text

In [17]:
# Apply the tokenizing_and_stopwords function to the "Cleaned_Text" column
B_types['ctns_Text'] = B_types['Cleaned_Text'].apply(tokenizing_and_stopwords)  
B_types[['ctns_Text']] #ctns_Text = cleaned, tokenized, no stopwords

,ctns_Text
0,fuck bitch fuck lousy misleading wikipedia che...
1,rt ryanklang actionflickdoc hey look http co
2,hopefully one day editors wikipedia wake turf ...
3,deceptive little creature indeed assuming good...
4,lol smalljim u small penis jk lol
...,...
35195,fucking hate niggers spics jews minorities fuc...
35196,congratulations well suppose confess wrong poi...
35197,also gon na get dickface
35198,stop fucking harrassing


In [18]:
B_types['ctns_Text'][94]

'ian black caley seen ian black play irractic life endagering challenges dunfermline rangers players show actual fucking nutter played blackburn youth team still'

In [19]:
# lemmatization aims to reduce words to their base or root form.

def lemmatization_and_stopwords(text):
    if not text:
        return ""
    tokens = word_tokenize(text)
    clean_text = []
    lemmatizer = nltk.WordNetLemmatizer()
    for token in tokens:
        if token.lower() not in stopwords.words('english') and len(token) > 3:
            token = lemmatizer.lemmatize(token)
            clean_text.append(token)
    result_text = " ".join(clean_text)
    return result_text

In [20]:
# Apply lemmatization function to the "ctns_Text" column
B_types['lemm_text'] = B_types['ctns_Text'].apply(lemmatization_and_stopwords)
B_types[['lemm_text']]

,lemm_text
0,fuck bitch fuck lousy misleading wikipedia che...
1,ryanklang actionflickdoc look http
2,hopefully editor wikipedia wake turf rubbish l...
3,deceptive little creature indeed assuming good...
4,smalljim small penis
...,...
35195,fucking hate nigger spic jew minority fucking ...
35196,congratulation well suppose confess wrong poin...
35197,also dickface
35198,stop fucking harrassing


In [21]:
# To show all the 'processed text' columns
B_types

,Text,oh_label,type_label,Cleaned_Text,ctns_Text,lemm_text
0,fuck you bitch. and fuck you lousy and misle...,1.0,aggression,fuck you bitch and fuck you lousy and misleadi...,fuck bitch fuck lousy misleading wikipedia che...,fuck bitch fuck lousy misleading wikipedia che...
1,RT @ryanklang: @Norse_Gamer @MT8_9 @ActionFlic...,1.0,sexism,rt ryanklang actionflickdoc hey look me too ht...,rt ryanklang actionflickdoc hey look http co,ryanklang actionflickdoc look http
2,::Hopefully one day the editors of Wikipedia ...,1.0,toxicity,hopefully one day the editors of wikipedia wil...,hopefully one day editors wikipedia wake turf ...,hopefully editor wikipedia wake turf rubbish l...
3,::::What a deceptive little creature you are...,1.0,aggression,what a deceptive little creature you are indee...,deceptive little creature indeed assuming good...,deceptive little creature indeed assuming good...
4,lol smalljim. u have small penis. jk lol,1.0,attack,lol smalljim u have small penis jk lol,lol smalljim u small penis jk lol,smalljim small penis
...,...,...,...,...,...,...
35195,I FUCKING HATE NIGGERS AND SPICS AND JEWS AND...,1.0,toxicity,i fucking hate niggers and spics and jews and ...,fucking hate niggers spics jews minorities fuc...,fucking hate nigger spic jew minority fucking ...
35196,` == Congratulations == Well I suppose I hav...,1.0,aggression,congratulations well i suppose i have to confe...,congratulations well suppose confess wrong poi...,congratulation well suppose confess wrong poin...
35197,"== You too! == You're also gonna get it, di...",1.0,attack,you too you re also gonna get it dickface,also gon na get dickface,also dickface
35198,Stop fucking harrassing me!,1.0,attack,stop fucking harrassing me,stop fucking harrassing,stop fucking harrassing


### Splitting the Dataset - X, y

In [22]:
X = B_types['lemm_text']
y = B_types[['type_label']] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the lengths of the resulting sets
print("Training set length:", len(X_train), len(y_train))
print("Testing set length:", len(X_test), len(y_test))

Training set length: 28160 28160
Testing set length: 7040 7040


### Feature Extraction

In [ ]:
### Feature Extraction: Extracting relevant features from the text data. 
# Using word embeddingsb (Word2Vec, GloVe) to capture semantic relationships between words

In [23]:
sentences = B_types['lemm_text']
model = Word2Vec(sentences, vector_size=500, window=5, min_count=1, workers=4)
#model.save("class_word2vec.model")

In [24]:
X_train.fillna("", inplace=True)
X_test.fillna("", inplace=True)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have a consistent length
max_sequence_length = 500  # You can adjust this based on your data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

print("Training set shape:", X_train_padded.shape, y_train.shape)
print("Testing set shape:", X_test_padded.shape, y_test.shape)

# Load pre-trained Word2Vec model
word2vec_model = Word2Vec.load("class_word2vec.model")

# Create an embedding matrix using Word2Vec
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, word2vec_model.vector_size))

for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Save the tokenizer
#joblib.dump(tokenizer, 'class_tokenizer.joblib') 

Training set shape: (28160, 500) (28160, 1)
Testing set shape: (7040, 500) (7040, 1)


In [25]:
B_types['type_label'].unique()

array(['aggression', 'sexism', 'toxicity', 'attack', 'racism'],
      dtype=object)

#### TENSORFLOW

In [26]:
# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert numerical labels to one-hot encoded vectors
y_train_one_hot = to_categorical(y_train_encoded, num_classes=5)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=5)

In [27]:
tf.keras.backend.clear_session()

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=word2vec_model.vector_size, 
                    input_length=max_sequence_length))
model.add(LSTM(units=100))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(X_train_padded, 
          y_train_one_hot, 
          epochs=3, 
          batch_size=32, 
          validation_data=(X_test_padded, y_test_one_hot))

Epoch 1/3
880/880 [==============================] - 946s 1s/step - loss: 1.1005 - accuracy: 0.3868 - val_loss: 1.0164 - val_accuracy: 0.4128
Epoch 2/3
880/880 [==============================] - 1109s 1s/step - loss: 0.9776 - accuracy: 0.4313 - val_loss: 1.0466 - val_accuracy: 0.3884
Epoch 3/3
880/880 [==============================] - 991s 1s/step - loss: 0.9431 - accuracy: 0.4648 - val_loss: 1.0882 - val_accuracy: 0.3209


In [32]:
evaluation_result = model.evaluate(X_test_padded, y_test_one_hot)

# Print the evaluation result
print("Test Loss:", evaluation_result[0])
print("Test Accuracy:", evaluation_result[1])

220/220 [==============================] - 55s 252ms/step - loss: 1.0882 - accuracy: 0.3209
Test Loss: 1.0882054567337036
Test Accuracy: 0.3208806812763214


In [34]:
# Make predictions with the model
y_pred_train_prob = model.predict(X_train_padded)
y_pred_test_prob = model.predict(X_test_padded)

# Convert probabilities to class labels
y_pred_train = np.argmax(y_pred_train_prob, axis=1)
y_pred_test = np.argmax(y_pred_test_prob, axis=1)

# Calculate performance metrics
performance_tf = pd.DataFrame({'Error_metric': ['Accuracy', 'Precision', 'Recall'],
                             'Train': [accuracy_score(y_train, y_pred_train),
                                       precision_score(y_train, y_pred_train, average='weighted', zero_division=1),
                                       recall_score(y_train, y_pred_train, average='weighted', zero_division=1)],
                             'Test': [accuracy_score(y_test, y_pred_test),
                                      precision_score(y_test, y_pred_test, average='weighted', zero_division=1),
                                      recall_score(y_test, y_pred_test, average='weighted', zero_division=1)]})


display(performance_tf)

220/220 [==============================] - 65s 296ms/step


ValueError: Mix of label input types (string and number)

In [ ]:
# Map class indices to class labels
y_pred_train_labels = [class_labels[idx] for idx in y_pred_train]
y_pred_test_labels = [class_labels[idx] for idx in y_pred_test]

In [ ]:
# Save the model in SavedModel format
#model.save('tf3_model') ### for classification